In [7]:
# Comandos Docker

# docker run -d --name postgres -p 5432:5432 -e POSTGRES_USER=dnc -e POSTGRES_PASSWORD=dnc123 debezium/postgres:12

In [8]:
# docker exec -ti postgres /bin/bash 

In [2]:
# Dentro do Container rodar os comandos

# cat /var/lib/postgresql/data/postgresql.conf

## Confirmar a configuracao 
### WAL_LEVEL = LOGICAL (record more data)
### MAX_WAL_SENDERS = 4 (numero de processos que Postgres pode rodar para enviar dados para o consumer)

# exit (para sair do container)

In [ ]:
# Utilizando o Dbeaver ou software similar para acessar o banco de dados Postgres

# Conectar no Banco de Dados

# Rodar os Comandos
# CREATE SCHEMA bank;
# SET search_path to bank, public;
# CREATE TABLE bank.holding (holding_id int, user_id int, holding_stock varchar(8), holding_quantity int, datetime_created timestamp, datetime_updated timestamp, primary key(holding_id));
# ALTER TABLE bank.holding replica identity FULL; 

# insert into bank.holding values (1000, 1, 'VFIAX', 10, now(), now())

In [11]:
# Iniciando Kafka com Docker

# docker run -d --name zookeeper -p 2181:2181 -p 2888:2888 -p 3888:3888 debezium/zookeeper:1.1   

# docker run -d --name kafka -p 9092:9092 --link zookeeper:zookeeper debezium/kafka:1.1     

In [13]:
# Iniciando Debezium com Docker

# docker run -d --name connect -p 8083:8083 --link kafka:kafka --link postgres:postgres -e BOOTSTRAP_SERVERS=kafka:9092 -e GROUP_ID=sde_group -e CONFIG_STORAGE_TOPIC=sde_storage_topic -e OFFSET_STORAGE_TOPIC=sde_offset_topic debezium/connect:1.1

# Verificando quais conexoes estao ativas
# curl -H "Accept:application/json" localhost:8083/connectors/ 

# Conectando nossa tabela que acabamos de criar   
# curl -H "Accept:application/json" -H "Content-Type: application/json" localhost:8083/connectors/ -d '{"name": "sde-connector", "config": {"connector.class": "io.debezium.connector.postgresql.PostgresConnector", "database.hostname": "postgres", "database.port": "5432", "database.user": "dnc", "database.password": "dnc123", "database.dbname": "postgres", "database.server.name": "bankserver1", "table.whitelist": "bank.holding"}}'

In [6]:
# Verificando os conectores

# curl -H "Accept:application/json" localhost:8083/connectors/   

In [20]:
# Consumer

# Lendo a primeira mensagem do topico

# docker run -it --rm --name consumer --link zookeeper:zookeeper --link kafka:kafka debezium/kafka:1.1 watch-topic -a bankserver1.bank.holding --max-messages 1  | grep '^{'

In [ ]:
# Para fazer o arquivo ser executavel

## chmod +x consumer.py  

In [10]:
# docker run -it --rm --name consumer --link zookeeper:zookeeper --link kafka:kafka debezium/kafka:1.1 watch-topic -a bankserver1.bank.holding  | grep --line-buffered '^{' | ./consumer.py > ./data/holding_pivot.txt

In [1]:
# comandos para rodar no PG database;

# insert into bank.holding values (10015, 10, 'NVDA', 40, now(), now());
# insert into bank.holding values (10025, 100, 'APPL', 30, now(), now());

# update bank.holding set holding_quantity = 100 where holding_id = 10015;

# delete from bank.holding where holding_id = 10015;
# delete from bank.holding where holding_id = 10025;

# insert into bank.holding values (10033, 2, 'APPL', 100, now(), now())

# update bank.holding set holding_quantity = 10 where holding_id = 10033;

In [ ]:
## PARAR OS CONTAINERS

# docker stop $(docker ps -aq)
# docker rm $(docker ps -aq)